In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
from keras.layers import LSTM,Dense
from keras.models import Sequential
from keras.utils import plot_model
from scipy.stats import uniform, truncnorm, randint

In [ ]:
train  = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/train_tweets.csv')
test = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/test_tweets.csv')

In [ ]:
test['tweet_topic_count'] = test.tweet_topic_ids.apply(len)
train['tweet_topic_count'] = train.tweet_topic_ids.apply(len)

In [ ]:
print(train.shape)
print(test.shape)

(29625, 14)
(12697, 13)


In [ ]:
train.head()

,tweet_id,tweet_user_id,tweet_created_at_year,tweet_created_at_month,tweet_created_at_day,tweet_created_at_hour,tweet_hashtag_count,tweet_url_count,tweet_mention_count,tweet_has_attachment,tweet_attachment_class,tweet_language_id,tweet_topic_ids,virality
0,34698,10,2015,12,5,3,2.0,1.0,0.0,False,C,0,"['36', '36', '36', '36', '36', '36', '37', '37...",3
1,24644,4,2020,6,19,0,0.0,1.0,0.0,False,C,0,"['43', '78', '79', '80', '80', '89', '98', '99...",3
2,36321,54,2019,6,2,15,2.0,3.0,0.0,True,A,0,"['79', '80', '98', '98', '98', '99', '99', '10...",1
3,2629,42,2020,9,6,17,0.0,1.0,1.0,True,A,0,"['43', '79', '80', '98', '99', '99', '79', '80']",2
4,28169,32,2020,11,4,17,2.0,1.0,0.0,True,A,0,"['79', '80', '98', '99', '43', '89']",2


# Attachment Class

In [ ]:
train.tweet_has_attachment.value_counts()

True     17800
False    11825
Name: tweet_has_attachment, dtype: int64

In [ ]:
test.tweet_has_attachment.value_counts()

True     7621
False    5076
Name: tweet_has_attachment, dtype: int64

In [ ]:
train.tweet_attachment_class.value_counts()

A    17511
C    11825
B      289
Name: tweet_attachment_class, dtype: int64

In [ ]:
test.tweet_attachment_class.value_counts()

A    7505
C    5076
B     116
Name: tweet_attachment_class, dtype: int64

In [ ]:
test_images = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/test_tweets_vectorized_media.csv')
train_images = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/train_tweets_vectorized_media.csv')

In [ ]:
test_images.head()

In [ ]:
test_images.shape

(8946, 2050)

In [ ]:
len(test_images.tweet_id.unique())

7475

In [ ]:
test_images.tweet_id.value_counts().value_counts()

1    6691
2     341
4     244
3     199
Name: tweet_id, dtype: int64

In [ ]:
train_images.tweet_id.value_counts().value_counts()

1    15510
2      834
4      556
3      536
Name: tweet_id, dtype: int64

In [ ]:
train_images.shape

(21010, 2050)

In [ ]:
len(train_images.tweet_id.unique())

17436

In [ ]:
y = train.pop('virality')


In [ ]:
train.tweet_attachment_class.map({'A':1})

#Topic ID

In [ ]:
train.tweet_topic_ids.value_counts()

['78', '79', '80']                                                                              4062
['78', '79', '80', '98', '99', '100', '78', '79', '80']                                         1539
['78', '79', '80', '98']                                                                        1136
['79', '80']                                                                                     735
['78', '79', '80', '98', '99', '100']                                                            650
                                                                                                ... 
['39', '43', '59', '59', '59', '93']                                                               1
['43', '93', '78', '78', '79', '79', '80', '80']                                                   1
['43', '79', '80', '98', '99', '163', '98', '98', '99', '99', '100', '100']                        1
['79', '80', '100', '98', '99', '99', '112']                                               

In [ ]:
test.tweet_topic_ids.head(5)

0    [98, 99, 99, 99, 99, 78, 79, 80]
1                    [78, 79, 80, 98]
2                        [78, 79, 80]
3                        [78, 79, 80]
4            [78, 79, 79, 80, 98, 99]
Name: tweet_topic_ids, dtype: object

In [ ]:
test.tweet_topic_ids = test.fillna('[]').tweet_topic_ids.apply(eval)
train.tweet_topic_ids = train.fillna('[]').tweet_topic_ids.apply(eval)

In [ ]:
type(train.tweet_topic_ids[0])

list

In [ ]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [ ]:
to_1D(train.tweet_topic_ids).unique()

array(['36', '37', '43', '78', '79', '80', '89', '98', '99', '100', '39',
       '44', '59', '156', '81', '93', '125', '163', '118', '87', '88',
       '152', '120', '45', '73', '82', '148', '121', '151', '63', '104',
       '111', '119', '91', '149', '147', '72', '60', '61', '170', '68',
       '58', '171', '155', '153', '112', '150', '169', '172', '52', '127',
       '122', '126', '101', '71', '165'], dtype=object)

In [ ]:
to_1D(test.tweet_topic_ids).unique()

array(['98', '99', '78', '79', '80', '100', '156', '43', '81', '87', '88',
       '36', '37', '44', '104', '171', '152', '61', '59', '93', '119',
       '120', '68', '45', '89', '63', '153', '169', '91', '39', '73',
       '163', '111', '60', '58', '112', '151', '147', '172', '118', '150',
       '170', '121', '127', '148', '72', '126', '122', '101', '149',
       '125', '82', '71', '38', '52', '155', '123', '117'], dtype=object)

In [ ]:
topics = set(to_1D(test.tweet_topic_ids).unique().tolist()+to_1D(train.tweet_topic_ids).unique().tolist())

In [ ]:
test['tweet_topic_count'] = test.tweet_topic_ids.apply(len)
train['tweet_topic_count'] = train.tweet_topic_ids.apply(len)

In [ ]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [ ]:
train_topics_bool = boolean_df(train['tweet_topic_ids'], topics)
test_topics_bool = boolean_df(test['tweet_topic_ids'], topics)

In [ ]:
test = pd.concat([test, test_topics_bool],axis=1)
train = pd.concat([train, train_topics_bool], axis=1)

In [ ]:
test.tweet_attachment_class = test.tweet_attachment_class.map({'A':1,'B':2,'C':3})
train.tweet_attachment_class = train.tweet_attachment_class.map({'A':1,'B':2,'C':3})

In [ ]:
train.pop('tweet_topic_ids')
test.pop('tweet_topic_ids')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20737, 83)
(8888, 83)
(20737,)
(8888,)


In [ ]:
train.shape

(29625, 83)

In [ ]:
test.shape

(12697, 83)

In [ ]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=25, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.8381154458214786
0.6828307830783078


# 

In [ ]:
clf = RandomForestClassifier(n_estimators=1200,max_depth=25, min_samples_leaf=1, min_samples_split=10)
clf.fit(train,y)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.8386941216183633
0.8421467146714672


In [ ]:
sol9 = test.tweet_id.to_frame()
sol9['virality'] = clf.predict(test)
sol9.astype(int).to_csv('sol11.csv',index=False)

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Users/users.csv')

In [ ]:
train = pd.concat([train,pd.DataFrame(columns=users.columns)])
test = pd.concat([test,pd.DataFrame(columns=users.columns)])
for i in range(train.shape[0]):
  train.loc[i,'user_id':'user_created_at_month'] = users[users.user_id==train.tweet_user_id[i]].iloc[0]
for i in range(test.shape[0]):
  test.loc[i,'user_id':'user_created_at_month'] = users[users.user_id==test.tweet_user_id[i]].iloc[0]

In [ ]:
train.head()

,tweet_id,tweet_user_id,tweet_created_at_year,tweet_created_at_month,tweet_created_at_day,tweet_created_at_hour,tweet_hashtag_count,tweet_url_count,tweet_mention_count,tweet_has_attachment,tweet_attachment_class,tweet_language_id,tweet_topics,80,89,60,156,59,100,148,44,171,111,39,58,52,147,125,122,123,63,165,152,72,82,71,150,117,36,87,...,153,93,121,169,91,151,172,127,149,98,68,61,170,81,38,155,73,45,104,43,37,163,118,79,119,120,88,78,112,user_id,user_like_count,user_followers_count,user_following_count,user_listed_on_count,user_has_location,user_tweet_count,user_has_url,user_verified,user_created_at_year,user_created_at_month
0,34698.0,10.0,2015.0,12.0,5.0,3.0,2.0,1.0,0.0,False,3.0,0.0,12.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,10,9365,398591,136081,18462,True,169980,True,0,2011,10
1,24644.0,4.0,2020.0,6.0,19.0,0.0,0.0,1.0,0.0,False,3.0,0.0,9.0,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,4,15311,459083,1021,7368,False,121193,False,0,2011,6
2,36321.0,54.0,2019.0,6.0,2.0,15.0,2.0,3.0,0.0,True,1.0,0.0,11.0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,54,336,447110,584,10519,True,3786,True,0,2011,12
3,2629.0,42.0,2020.0,9.0,6.0,17.0,0.0,1.0,1.0,True,1.0,0.0,8.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,42,1124,276541,758,6163,True,3264,True,0,2011,9
4,28169.0,32.0,2020.0,11.0,4.0,17.0,2.0,1.0,0.0,True,1.0,0.0,6.0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,32,0,1587783,2,16414,False,2022,True,0,2013,7


In [ ]:
forest = RandomForestClassifier(random_state = 1)
modelF = forest.fit(X_train, y_train)
print('Train Accuracy: ', forest.score(X_train,y_train))
print('Test Accuracy: ', forest.score(X_test,y_test))

Train Accuracy:  1.0
Test Accuracy:  0.6748424842484249


In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = RandomizedSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1,)
bestF = gridF.fit(X_train, y_train)

In [ ]:
model_params = {
    # randomly sample numbers from 4 to 204 estimators
    'n_estimators': randint(4,1400),
    # normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    'min_samples_split': uniform(0.01, 0.299)
}

In [ ]:
clf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=25, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# this will train 100 models over 5 folds of cross validation (500 models total)
random_search = RandomizedSearchCV(clf, model_params, n_iter=100, cv=5, random_state=1)

# train the random search meta-estimator to find the best model out of 100 candidates
model = random_search.fit(X_train, y_train)


In [ ]:
model.best_score_

0.6487920768089025

In [ ]:
model.best_params_

{'max_features': 0.26750178061074575,
 'min_samples_split': 0.01239281048497777,
 'n_estimators': 509}

In [ ]:
clf2 = RandomForestClassifier(max_features=0.26345595027850155, min_samples_split=0.013696664233304495, n_estimators=170)
clf2.fit(train,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None,
                       max_features=0.26345595027850155, max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=0.013696664233304495,
                       min_weight_fraction_leaf=0.0, n_estimators=170,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
clf = RandomForestClassifier(n_estimators=170, max_features=0.26345595027850155, min_samples_split=0.013696664233304495)
clf.fit(X_train,y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.6546269952259247
0.6586408640864087


In [ ]:
sol9 = test.tweet_id.to_frame()
sol9['virality'] = clf.predict(test)
sol9.astype(int).to_csv('sol12.csv',index=False)